In [3]:
import pandas as pd
import numpy as np
import requests
import yfinance as yf
import matplotlib.pyplot as plt
from math import floor
from termcolor import colored as cl

plt.style.use("fivethirtyeight")
plt.rcParams["figure.figsize"] = (20, 10)

In [4]:
aapl = yf.download("AAPL", start="2015-01-01", end="2020-01-01").reset_index()

/home/jupyter/.cache/pypoetry/virtualenvs/compare-multiple-backtests-2J6ybkfi-py3.10/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [6]:
def get_stoch_osc(high, low, close, k_lookback, d_lookback):
    lowest_low = low.rolling(k_lookback).min()
    highest_high = high.rolling(k_lookback).max()
    k_line = ((close - lowest_low) / (highest_high - lowest_low)) * 100
    d_line = k_line.rolling(d_lookback).mean()
    return k_line, d_line


aapl["%k"], aapl["%d"] = get_stoch_osc(aapl["High"], aapl["Low"], aapl["Close"], 14, 3)
aapl.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,%k,%d
1253,2019-12-24,71.172501,71.222504,70.730003,71.067497,69.148010,48478800,97.202051,94.654622
1254,2019-12-26,71.205002,72.495003,71.175003,72.477501,70.519897,93121200,99.720753,98.620711
1255,2019-12-27,72.779999,73.492500,72.029999,72.449997,70.493164,146266000,85.650331,94.191045
1256,2019-12-30,72.364998,73.172501,71.305000,72.879997,70.911552,144114400,91.284202,92.218429
1257,2019-12-31,72.482498,73.419998,72.379997,73.412498,71.429672,100805600,98.799222,91.911252


In [8]:
def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span=fast, adjust=False).mean()
    exp2 = price.ewm(span=slow, adjust=False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns={"Close": "macd"})
    signal = pd.DataFrame(macd.ewm(span=smooth, adjust=False).mean()).rename(
        columns={"macd": "signal"}
    )
    hist = pd.DataFrame(macd["macd"] - signal["signal"]).rename(columns={0: "hist"})
    return macd, signal, hist


aapl["macd"] = get_macd(aapl["Close"], 26, 12, 9)[0]
aapl["macd_signal"] = get_macd(aapl["Close"], 26, 12, 9)[1]
aapl["macd_hist"] = get_macd(aapl["Close"], 26, 12, 9)[2]
aapl = aapl.dropna()
aapl.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,%k,%d,macd,macd_signal,macd_hist
1253,2019-12-24,71.172501,71.222504,70.730003,71.067497,69.148010,48478800,97.202051,94.654622,1.656653,1.501481,0.155172
1254,2019-12-26,71.205002,72.495003,71.175003,72.477501,70.519897,93121200,99.720753,98.620711,1.781618,1.557508,0.224109
1255,2019-12-27,72.779999,73.492500,72.029999,72.449997,70.493164,146266000,85.650331,94.191045,1.857027,1.617412,0.239615
1256,2019-12-30,72.364998,73.172501,71.305000,72.879997,70.911552,144114400,91.284202,92.218429,1.929248,1.679779,0.249469
1257,2019-12-31,72.482498,73.419998,72.379997,73.412498,71.429672,100805600,98.799222,91.911252,2.006325,1.745089,0.261236


In [12]:
def implement_stoch_macd_strategy(prices, k, d, macd, macd_signal):
    buy_price = []
    sell_price = []
    stoch_macd_signal = []
    signal = 0

    for i in range(len(prices)):
        if k[i] < 30 and d[i] < 30 and macd[i] < -2 and macd_signal[i] < -2:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                stoch_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_macd_signal.append(0)

        elif k[i] > 70 and d[i] > 70 and macd[i] > 2 and macd_signal[i] > 2:
            if signal != -1 and signal != 0:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                stoch_macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stoch_macd_signal.append(0)

        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            stoch_macd_signal.append(0)

    return buy_price, sell_price, stoch_macd_signal


aapl = aapl.reset_index(drop=True)
buy_price, sell_price, stoch_macd_signal = implement_stoch_macd_strategy(
    aapl["Close"], aapl["%k"], aapl["%d"], aapl["macd"], aapl["macd_signal"]
)

In [14]:
position = []
for i in range(len(stoch_macd_signal)):
    if stoch_macd_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)

for i in range(len(aapl["Close"])):
    if stoch_macd_signal[i] == 1:
        position[i] = 1
    elif stoch_macd_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i - 1]

close_price = aapl["Close"]
k_line = aapl["%k"]
d_line = aapl["%d"]
macd_line = aapl["macd"]
signal_line = aapl["macd_signal"]
stoch_macd_signal = (
    pd.DataFrame(stoch_macd_signal)
    .rename(columns={0: "stoch_macd_signal"})
    .set_index(aapl.index)
)
position = (
    pd.DataFrame(position)
    .rename(columns={0: "stoch_macd_position"})
    .set_index(aapl.index)
)

frames = [
    close_price,
    k_line,
    d_line,
    macd_line,
    signal_line,
    stoch_macd_signal,
    position,
]
strategy = pd.concat(frames, join="inner", axis=1)

strategy

,Close,%k,%d,macd,macd_signal,stoch_macd_signal,stoch_macd_position
0,28.275000,87.050342,89.582210,0.156891,0.036015,0,1
1,27.285000,43.013110,73.873503,0.120245,0.052861,0,1
2,28.827499,78.250744,69.438065,0.213212,0.084931,0,1
3,29.725000,97.927085,73.063646,0.355215,0.138988,0,1
4,29.290001,80.810840,85.662889,0.427722,0.196735,0,1
...,...,...,...,...,...,...,...
1238,71.067497,97.202051,94.654622,1.656653,1.501481,0,0
1239,72.477501,99.720753,98.620711,1.781618,1.557508,0,0
1240,72.449997,85.650331,94.191045,1.857027,1.617412,0,0
1241,72.879997,91.284202,92.218429,1.929248,1.679779,0,0


In [15]:
aapl_ret = pd.DataFrame(np.diff(aapl["Close"])).rename(columns={0: "returns"})
stoch_macd_strategy_ret = []

for i in range(len(aapl_ret)):
    try:
        returns = aapl_ret["returns"][i] * strategy["stoch_macd_position"][i]
        stoch_macd_strategy_ret.append(returns)
    except:
        pass

stoch_macd_strategy_ret_df = pd.DataFrame(stoch_macd_strategy_ret).rename(
    columns={0: "stoch_macd_returns"}
)

investment_value = 100000
number_of_stocks = floor(investment_value / aapl["Close"][0])
stoch_macd_investment_ret = []

for i in range(len(stoch_macd_strategy_ret_df["stoch_macd_returns"])):
    returns = number_of_stocks * stoch_macd_strategy_ret_df["stoch_macd_returns"][i]
    stoch_macd_investment_ret.append(returns)

stoch_macd_investment_ret_df = pd.DataFrame(stoch_macd_investment_ret).rename(
    columns={0: "investment_returns"}
)
total_investment_ret = round(sum(stoch_macd_investment_ret_df["investment_returns"]), 2)
profit_percentage = floor((total_investment_ret / investment_value) * 100)
print(
    cl(
        "Profit gained from the STOCH MACD strategy by investing $100k in AAPL : {}".format(
            total_investment_ret
        ),
        attrs=["bold"],
    )
)
print(
    cl(
        "Profit percentage of the STOCH MACD strategy : {}%".format(profit_percentage),
        attrs=["bold"],
    )
)

Profit gained from the STOCH MACD strategy by investing $100k in AAPL : 131592.23
Profit percentage of the STOCH MACD strategy : 131%
